# Preciador de vehículos

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

### Inicialización

In [30]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

### Carga de datos

In [18]:
try:
    df = pd.read_csv('/datasets/car_data.csv')
except:
    df = pd.read_csv('datasets/car_data.csv')

### Exploración de datos

In [19]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [21]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Se cambian los valores no y yes de la columna NotRepaired por valores 0 y 1 para poderlo usar para el entrenamiento del modelo.

In [22]:
df['NotRepaired'] = df['NotRepaired'].replace('no',0)
df['NotRepaired'] = df['NotRepaired'].replace('yes',1)
df['NotRepaired'] = df['NotRepaired'].fillna(0)

C:\Users\reiva\AppData\Local\Temp\ipykernel_21612\1297644283.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['NotRepaired'] = df['NotRepaired'].replace('yes',1)


In [23]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,0.0,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,1.0,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,0.0,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,0.0,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,0.0,31/03/2016 00:00,0,60437,06/04/2016 10:17


Verificamos la cantidad de valores ausentes.

In [24]:
na_ratio = ((df.isnull().sum() / len(df))*100).sort_values(ascending = False)
na_ratio

VehicleType          10.579368
FuelType              9.282697
Gearbox               5.596709
Model                 5.560588
DateCrawled           0.000000
Price                 0.000000
RegistrationYear      0.000000
Power                 0.000000
Mileage               0.000000
RegistrationMonth     0.000000
Brand                 0.000000
NotRepaired           0.000000
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

Observamos que todas las columnas donde hay valores ausentes son columnas con valores cualitativos que no afectan el entrenamiento del modelo, por tanto, no se imputan los valores ausentes.

Verificamos la cantidad de valores duplicados.

In [25]:
df.duplicated().sum()

291

Encontramos valores duplicados y para no alterar el entrenamiento del modelo se decide eliminar los valores duplicados. Adicionalmente, son muy pocos valores ausentes comparados con el total de filas del dataframe.

In [26]:
df = df.drop_duplicates()

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354078 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DateCrawled        354078 non-null  object 
 1   Price              354078 non-null  int64  
 2   VehicleType        316594 non-null  object 
 3   RegistrationYear   354078 non-null  int64  
 4   Gearbox            334248 non-null  object 
 5   Power              354078 non-null  int64  
 6   Model              334377 non-null  object 
 7   Mileage            354078 non-null  int64  
 8   RegistrationMonth  354078 non-null  int64  
 9   FuelType           321189 non-null  object 
 10  Brand              354078 non-null  object 
 11  NotRepaired        354078 non-null  float64
 12  DateCreated        354078 non-null  object 
 13  NumberOfPictures   354078 non-null  int64  
 14  PostalCode         354078 non-null  int64  
 15  LastSeen           354078 non-null  object 
dtypes: floa

## Entrenamiento del modelo 

Seleccionamos las columnas que conformarán las características y el objetivo

In [37]:
features = df[['RegistrationYear','Power','Mileage','NotRepaired']]
target = df['Price']

Dividimos el dataframe en entrenamiento y test en una proporción 70:30

In [38]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=42)

### Regresión lineal

In [41]:
lr = LinearRegression()
lr.fit(features_train, target_train)
pred_lr = lr.predict(features_test)

### Bosque aleatorio

In [44]:
rf = RandomForestRegressor(random_state=42)

# Definimos la grilla de hiperparámetros a probar
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
}

grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='neg_mean_squared_error')
grid_search_rf.fit(features_train, target_train)

pred_rf = grid_search_rf.predict(features_test)

KeyboardInterrupt: 

### LightGBM

In [45]:
lgbm = LGBMRegressor(random_state=42)

# Definimos la grilla de hiperparámetros a probar
param_grid_lgbm = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.01],
}

grid_search_lgbm = GridSearchCV(lgbm, param_grid_lgbm, cv=3, scoring='neg_mean_squared_error')
grid_search_lgbm.fit(features_train, target_train)

pred_lgbm = grid_search_lgbm.predict(features_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 368
[LightGBM] [Info] Number of data points in the train set: 165236, number of used features: 4
[LightGBM] [Info] Start training from score 4406.960971
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000931 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 368
[LightGBM] [Info] Number of data points in the train set: 165236, number of used features: 4
[LightGBM] [Info] Start training from score 4424.152588
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

## Análisis del modelo

### Regresión lineal

In [42]:
rmse_lr = mean_squared_error(target_test, pred_lr,squared=False)
print(f'RMSE para Regresión Lineal: {rmse_lr}')

RMSE para Regresión Lineal: 4147.074343582099


### Bosque aleatorio

In [ ]:
rmse_rf = mean_squared_error(target_test, pred_rf, squared=False)
print(f'RMSE para Bosque Aleatorio: {rmse_rf}')

### LightGBM

In [46]:
rmse_lgbm = mean_squared_error(target_test, pred_lgbm, squared=False)
print(f'RMSE para LightGBM: {rmse_lgbm}')

RMSE para LightGBM: 2159.763832413146


c:\Users\reiva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos